# Custom spider v0.1
Second generation of custom spider, used to crawl cookbooks.com  
Belive me or not, this shitty code managed to get almost 1 milion recipes.  

In [1]:
import urllib.request

import sys
import time
import random
import json

from bs4 import BeautifulSoup

In [2]:
folder = 'cookbooks'
url = 'http://www.cookbooks.com/Recipe-Details.aspx?id='
last = 1086415

In [3]:
# get test page
html_doc = urllib.request.urlopen('http://www.cookbooks.com/Recipe-Details.aspx?id='+str(1)).read().decode('utf-8')
soup = BeautifulSoup(html_doc, 'html.parser')

In [4]:
tags = soup.find_all('span', 'H2')
tags

[<span class="H2"><font size="4"><i></i></font></span>,
 <span class="H2">ingredients</span>,
 <span class="H2">preparation</span>,
 <span class="H2">Recipe Comments</span>]

In [5]:
tags[2].parent.p.get_text('\n').strip()

'Combine crumbs, nuts, 3 tablespoons granulated sugar and margarine.  Press onto bottom of 9-inch spring-form pan.  Bake at 325° for 10 minutes.  Combine cream cheese and brown sugar, mixing at medium speed with electric mixer until well blended.  Add eggs, one at a time, mixing well after each.  Blend in chocolate, milk, and vanilla extract; pour over crust.  Bake at 325° for 35 minutes. Combine sour cream and 2 tablespoons granulated sugar; carefully spread over cheesecake.  Bake at 425° for 10 minutes.  Loosen cake from rim of pan.  Cool before removing rim of pan.  Chill.  Garnish with mint candies.'

In [6]:
soup.find_all('p', 'H1')

[<p class="H1">1  c. vanilla wafer crumbs<br/>1/2  c. chopped pecans<br/>3  Tbsp. granulated sugar<br/>1/2  c. Parkay margarine<br/>2 (8 oz.)  pkg. Philadelphia cream cheese, softened<br/>1/2  c. packed brown sugar<br/>2  eggs<br/>16  oz. pkg. semi-sweet chocolate pieces<br/>2  Tbsp. milk<br/>1  tsp. vanilla extract<br/>2  c. sour cream<br/>2  Tbsp. granulated sugar<br/>  mint candies (garnish)<br/></p>,
 <p class="H1">      Combine crumbs, nuts, 3 tablespoons granulated sugar and margarine.  Press onto bottom of 9-inch spring-form pan.  Bake at 325° for 10 minutes.  Combine cream cheese and brown sugar, mixing at medium speed with electric mixer until well blended.  Add eggs, one at a time, mixing well after each.  Blend in chocolate, milk, and vanilla extract; pour over crust.  Bake at 325° for 35 minutes. Combine sour cream and 2 tablespoons granulated sugar; carefully spread over cheesecake.  Bake at 425° for 10 minutes.  Loosen cake from rim of pan.  Cool before removing rim of pa

In [7]:
# find and parse ingredients
def ingredients_cookbooks(soup):
    ingredients_tag = soup.find_all('span', 'H2', string='ingredients')[0]
    # print(ingredients_tag)
    return ingredients_tag.parent.p.get_text('|').strip().split('|')

In [8]:
# find and parse directions
def directions_cookbooks(soup):
    directions_tag = soup.find_all('span', 'H2', string='preparation')[0]
    # print(directions_tag)
    return directions_tag.parent.p.get_text('|').strip().split('  ')

In [9]:
# title
def title_cookbooks(soup):
    return soup.find_all('p', 'H2')[0].get_text().lower()

In [10]:
def recipe_cookbooks(soup):
    ret = dict()
    ret['title'] = title_cookbooks(soup)
    ret['ingredients'] = ingredients_cookbooks(soup)
    ret['directions'] = directions_cookbooks(soup)
    return ret

In [11]:
def handle_recipe(soup, retrive_recipe, retrive_title, path=".", filename_prefix='', filename_suffix=''):
    title = retrive_title(soup)
    body = json.dumps(retrive_recipe(soup))
    # TODO check if such a file exists
    with open(path+'/'+filename_prefix+title.lower().replace(' ','_')+filename_suffix+'.json', 'w+') as f:
        f.write(body)

In [12]:
## the proper function to be used in future and in framework
def save_recipe(step, url, retrieve_ingredients, retrieve_directions, retrieve_title, path="./", filename_prefix="", filename_suffix=""):
    try:
        start = time.time()
        
        # get page
        html_doc = urllib.request.urlopen(url).read().decode('utf-8')
        # parse
        soup = BeautifulSoup(html_doc, 'html.parser')
        # retrive information and save to dictionary
        title = retrieve_title(soup)
        recipe = dict()
        recipe['title'] = title
        recipe['ingredients'] = retrieve_ingredients(soup)
        recipe['directions'] = retrieve_directions(soup)
        # save to file
        with open(path+'/'+filename_prefix+title.lower().replace(' ','_')+filename_suffix+'.json', 'w+') as f:
            f.write(json.dumps(recipe))
            
        end = time.time()
        elapsed = end-start
        
        print("Step:\t", step, "Time:\t", elapsed)
    
    except:
        print("Unable to get recipe from:\t", url)
        #for s in sys.exc_info():
        #    print(s)
        

In [13]:
recipes = []
recipes_count = 0
total = 0
missed = 0
counter = 0
for i in range(1,10):
    # print(i)
    counter += 1
    try:
        start = time.time()
        
        html_doc = urllib.request.urlopen('http://www.cookbooks.com/Recipe-Details.aspx?id='+str(i)).read().decode('utf-8')
        soup = BeautifulSoup(html_doc, 'html.parser')
        # recipes.append(recipe_cookbooks(soup))
        
        str_id = '0' * (8 - len(str(i))) + str(i) # since names/titles are not unique, id is added
        
        handle_recipe(soup, retrive_recipe=recipe_cookbooks, retrive_title=title_cookbooks, path='cookbooks/temp', filename_prefix=str_id+'-')
        
        end = time.time()
        elapsed = end-start
        total += elapsed
        
        recipes_count += 1
        
        print("Step:\t", i, "\tRecipes:\t", recipes_count, "\tTime:\t", elapsed, "\tMean time:\t", total/recipes_count, '\tMissing:\t', missed/counter)
        
    except KeyboardInterrupt:
        print("Finish and save")
        break
        
    except:
        print("Unable to get recipe with id:\t" + str(i))
        missed += 1

Step:	 1 	Recipes:	 1 	Time:	 1.4868807792663574 	Mean time:	 1.4868807792663574 	Missing:	 0.0
Unable to get recipe with id:	2
Step:	 3 	Recipes:	 2 	Time:	 1.280451774597168 	Mean time:	 1.3836662769317627 	Missing:	 0.3333333333333333
Unable to get recipe with id:	4
Step:	 5 	Recipes:	 3 	Time:	 1.4655299186706543 	Mean time:	 1.4109541575113933 	Missing:	 0.4
Step:	 6 	Recipes:	 4 	Time:	 1.321441888809204 	Mean time:	 1.388576090335846 	Missing:	 0.3333333333333333
Step:	 7 	Recipes:	 5 	Time:	 1.3532140254974365 	Mean time:	 1.3815036773681642 	Missing:	 0.2857142857142857
Step:	 8 	Recipes:	 6 	Time:	 1.954516887664795 	Mean time:	 1.4770058790842693 	Missing:	 0.25
Step:	 9 	Recipes:	 7 	Time:	 1.2735438346862793 	Mean time:	 1.4479398727416992 	Missing:	 0.2222222222222222


In [14]:
# test for long tasks
import multiprocessing as mp

In [15]:
#
# only downloads links from the list
#

sleep_time = 1
max_processes = 8
counter = 0

seed = 'http://www.cookbooks.com/Recipe-Details.aspx?id='

processes = []
for i in range(1, 10):
    # url selection or other custom shit here
    str_id = '0' * (8 - len(str(i))) + str(i) # since names/titles are not unique, id is added
    
    arguments = (i, seed+str(i), ingredients_cookbooks, directions_cookbooks, title_cookbooks, 'cookbooks/temp', str_id+'-')
    
    target_func = save_recipe
    
    ### try not to edit code below this line
    
    counter += 1
    inactive = []
    # visit list of processes
    for proc in processes:
        # when process is no longer active, join it and add to list of inactive processes
        if not proc.is_alive():
            proc.join()
            inactive.append(proc)
    # remove inactive processes from processes list
    while inactive:
        processes.remove(inactive.pop())
    # print("Number of active processes:\t", len(processes))
    
    # if number of active processes is acceptable, we can start new process
    if len(processes) < max_processes: 
        p = mp.Process(target = target_func, args = arguments)
        p.start()
        processes.append(p)
    else:
        print("List of processes is full")
    
    # sleep, to avoid ddos attack or to fit in robots.txt rules
    time.sleep(sleep_time)

# join remaining processes
while processes:
    temp = processes.pop()
    temp.join()
    

Step:	 1 Time:	 1.317464828491211
Unable to get recipe from:	 http://www.cookbooks.com/Recipe-Details.aspx?id=2
Step:	 3 Time:	 1.4252955913543701
Unable to get recipe from:	 http://www.cookbooks.com/Recipe-Details.aspx?id=4
Step:	 5 Time:	 1.318800687789917
Step:	 6 Time:	 1.5576882362365723
Step:	 7 Time:	 1.2769484519958496
Step:	 8 Time:	 1.291440725326538
Step:	 9 Time:	 1.4693281650543213


In [16]:
# find missing files
from os import listdir

In [17]:
filenames = listdir('cookbooks/recipes/')

In [18]:
filenames.remove('.ipynb_checkpoints')

In [19]:
last = 1086416

In [20]:
func = lambda x: int(x.split('-')[0])

In [21]:
func(filenames[0])

619430

In [22]:
ids = list(map(func, filenames))
ids.sort()

In [23]:
missing = []

In [24]:
ne = ids.pop(0)
for i in range(1, last):
    if len(ids) == 0 or i < ne:
        missing.append(i)
    elif i == ne:
        ne = ids.pop(0)

In [25]:
len(missing)

131741

In [26]:
with open('./cookbooks/missing.txt', 'w+') as f:
    for m in missing:
        f.write(str(m)+'\n')